In [1]:
import ast
import random

LANGUAGE = 'en'

In [2]:
import os

import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [3]:
os.getcwd()

'/cluster/scratch/yakram/sbert-copy/1_my_finetuning_scripts/mnr/data_creation'

In [4]:
CURRENT_FILE_LOCATION = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(CURRENT_FILE_LOCATION, '../../turtle_files')

In [37]:
CURRENT_FILE_LOCATION

'/cluster/scratch/yakram/sbert-copy/1_my_finetuning_scripts/mnr'

In [5]:
if LANGUAGE == 'en':
    DATA_FILES_PATH = os.path.join(DATA_DIR, 'english_data')
elif LANGUAGE == 'de':
    DATA_FILES_PATH = os.path.join(DATA_DIR, 'german_data')

NEGATIVE_FILE_PATH = '/cluster/scratch/yakram/sbert-copy/turtle_files/negative_100.csv'

In [6]:
MemConcept_prefLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_prefLabel.rq'), sep='\t', index_col=[0])
MemConcept_altLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_altLabel.rq'), sep='\t', index_col=[0])
MemConcept_hiddenLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_hiddenLabel.rq'), sep='\t', index_col=[0])
MemConcept_description = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_description.rq'), sep='\t', index_col=[0])

SkillsHier_prefLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_prefLabel.rq'), sep='\t', index_col=[0])
SkillsHier_altLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_altLabel.rq'), sep='\t', index_col=[0])
SkillsHier_hiddenLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_hiddenLabel.rq'), sep='\t', index_col=[0])
SkillsHier_description = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_description.rq'), sep='\t', index_col=[0])

MemConcept_SkillsHier_prefLabel = pd.concat([MemConcept_prefLabel, SkillsHier_prefLabel])
MemConcept_SkillsHier_altLabel = pd.concat([MemConcept_altLabel, SkillsHier_altLabel])
MemConcept_SkillsHier_hiddenLabel = pd.concat([MemConcept_hiddenLabel, SkillsHier_hiddenLabel])
MemConcept_SkillsHier_description = pd.concat([MemConcept_description, SkillsHier_description])

skills_uri = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_concept_schemes_combined.rq'), sep='\t', index_col=[0])
negative_file = pd.read_csv(NEGATIVE_FILE_PATH, sep='\t', index_col=[0])

negative_file.columns = ['?a', 'negative_uris']

skill_with_neg = negative_file.merge(skills_uri, on=['?a'], how='left')

In [7]:
skill_with_neg

,?a,negative_uris
0,http://data.europa.eu/esco/isced-f/00,['http://data.europa.eu/esco/skill/b74fd765-e1...
1,http://data.europa.eu/esco/isced-f/000,['http://data.europa.eu/esco/skill/b779205e-37...
2,http://data.europa.eu/esco/isced-f/0000,['http://data.europa.eu/esco/skill/1fe4c04b-30...
3,http://data.europa.eu/esco/isced-f/001,['http://data.europa.eu/esco/skill/22d363fa-45...
4,http://data.europa.eu/esco/isced-f/0011,['http://data.europa.eu/esco/skill/aef7eab0-ae...
...,...,...
14520,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,['http://data.europa.eu/esco/skill/8353579b-20...
14521,http://data.europa.eu/esco/skill/fff0b074-5a76...,['http://data.europa.eu/esco/skill/8ff015a0-98...
14522,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,['http://data.europa.eu/esco/skill/8c86584c-5a...
14523,http://data.europa.eu/esco/skill/fff5bc45-b506...,['http://data.europa.eu/esco/skill/5125b132-d1...


In [8]:
def create_double_pair(df1, df2):

    skills_copy = skills_uri.copy(deep=True)
    skills_copy = pd.DataFrame(skills_copy)

    combined_df = skills_copy.merge(df1, on=['?a'], how='left').merge(df2, on=['?a'], how='left')
    # combined_df.drop(columns=['?a'], inplace=True)
    return combined_df

In [9]:
pref_alt_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_altLabel)
pref_hidden_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_hiddenLabel)
pref_description_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_description)
alt_hidden_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_hiddenLabel)
alt_description_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_description)
hidden_description_data = create_double_pair(MemConcept_SkillsHier_hiddenLabel, MemConcept_SkillsHier_description)

In [10]:
pref_alt_data.dropna(inplace=True)
pref_hidden_data.dropna(inplace=True)
pref_description_data.dropna(inplace=True)
alt_hidden_data.dropna(inplace=True)
alt_description_data.dropna(inplace=True)
hidden_description_data.dropna(inplace=True)

In [11]:
all_positive_pairs = [pref_alt_data,pref_hidden_data,pref_description_data,alt_hidden_data,alt_description_data,hidden_description_data,]

In [12]:
total_original = 222942
total_now = 0
for pair_data in all_positive_pairs:
    print(pair_data.shape)
    total_now += pair_data.shape[0]

(97626, 3)
(1014, 3)
(14396, 3)
(25444, 3)
(97623, 3)
(1014, 3)


In [36]:
def sample_negatives(negative_col, count_col):

    return random.sample(ast.literal_eval(negative_col), k=count_col)

def sample_random_from_uri(random_sample_uri, df_containing_property):

    try:
        merged_df = df_containing_property[df_containing_property['?a'] == random_sample_uri]
        if merged_df.shape != 0:
            merged_df_grouped = merged_df.groupby(by=['?a']).sample(n=1).reset_index()[merged_df.columns[-1]].values

            return merged_df_grouped
        else:
            return 'no value found'
    except:
        return 'no value found'

In [18]:
def get_negatives_until_explosion(main_dataframe, matcher_df):

    copy_main_dataframe = main_dataframe.copy(deep=True)
    copy_matcher_df = matcher_df.copy(deep=True)
    copy_matcher_df.rename(columns={'?a': 'sampled_negatives'}, inplace=True)
    copy_matcher_df_random_sampled = copy_matcher_df.groupby(by='sampled_negatives').first().reset_index()

    main_data_counts = copy_main_dataframe.groupby(by=['?a']).count()[copy_main_dataframe.columns[1]].reset_index()
    main_data_counts.rename(columns={main_data_counts.columns[1]: 'counts'}, inplace=True)
    main_data_data_counts_merged = copy_main_dataframe.merge(main_data_counts, on=['?a'], how='left')
    main_data_data_counts_merged_negative = negative_file.merge(main_data_data_counts_merged, on=['?a'], how='left')
    main_data_data_counts_merged_negative_nan = main_data_data_counts_merged_negative.dropna()
    main_data_data_counts_merged_negative_nan['counts'] = main_data_data_counts_merged_negative_nan['counts'].apply(lambda x: int(x))
    main_data_data_counts_merged_negative_nan = main_data_data_counts_merged_negative_nan[main_data_data_counts_merged_negative_nan['counts'] <= 100]

    main_data_subset = main_data_data_counts_merged_negative_nan[['?a', 'negative_uris', 'counts']]
    main_data_subset.dropna(inplace=True)
    main_data_subset.drop_duplicates(inplace=True)

    main_data_subset['sampled_negatives'] = main_data_subset.progress_apply(lambda x: sample_negatives(x.negative_uris, x.counts), axis=1)
    main_data_subset.drop(columns=['negative_uris'], inplace=True)

    exploded_dataset = main_data_subset.explode('sampled_negatives')
    exploded_dataset.drop(columns=['counts'], inplace=True)


    exploded_dataset_merged_property = exploded_dataset.merge(copy_matcher_df_random_sampled, on=['sampled_negatives'], how='left')

    return exploded_dataset_merged_property

In [ ]:
raise

In [30]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

# df.apply(func)
# df.parallel_apply(func)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [31]:
exploded_dataset['altnegatives'] = exploded_dataset['sampled_negatives'].parallel_apply(lambda x: sample_random_from_uri(x, MemConcept_SkillsHier_altLabel))

/cluster/home/yakram/.pyenv/versions/3.8.2/envs/thesis/lib/python3.8/site-packages/pandarallel/data_types/series.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield data[chunk_]


In [ ]:
random_sample_uri = 'http://data.europa.eu/esco/skill/9c0148f3-838f-4a06-b929-e23d94535773'
df_containing_property = MemConcept_SkillsHier_altLabel

In [ ]:
merged_df = df_containing_property[df_containing_property['?a'] == random_sample_uri]

In [ ]:
merged_df_grouped = merged_df.groupby(by=['?a']).sample(n=1).reset_index()[merged_df.columns[-1]].values

In [ ]:
merged_df_grouped

In [ ]:


exploded_dataset_merged_property = exploded_dataset.merge(copy_matcher_df_random_sampled, on=['sampled_negatives'], how='left')


In [ ]:
exploded_dataset = main_data_subset.explode('sampled_negatives')

In [ ]:
exploded_dataset

In [ ]:


exploded_dataset.drop(columns=['counts'], inplace=True)


exploded_dataset_merged_property = exploded_dataset.merge(copy_matcher_df_random_sampled, on=['sampled_negatives'], how='left')


In [ ]:
exploded_dataset_merged_property.drop(columns=['sampled_negatives'], inplace=True)

In [ ]:
exploded_dataset_merged_property

In [ ]:
exploded_dataset_merged_property_grouped = exploded_dataset_merged_property.groupby(by=['?a']).first().reset_index()

In [ ]:
exploded_dataset_merged_property_grouped

In [ ]:
main_data_data_counts_merged_negative_nan

In [ ]:
fine_merged_negatives = main_data_data_counts_merged_negative_nan.merge(exploded_dataset_merged_property_grouped, on=['?a'], how='left')
fine_merged_negatives.shape

In [ ]:
fine_merged_negatives_triplets = fine_merged_negatives[['?prefLabel', '?altLabel_x', '?altLabel_y']]

In [ ]:
fine_merged_negatives_triplets

In [ ]:
pref_alt_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_altLabel)
pref_hidden_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_hiddenLabel)
pref_description_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_description)
alt_hidden_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_hiddenLabel)
alt_description_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_description)
hidden_description_data = create_double_pair(MemConcept_SkillsHier_hiddenLabel, MemConcept_SkillsHier_description)

In [ ]:
exploded_dataset

In [ ]:
pref_alt_negatives = get_negatives_until_explosion(pref_alt_data)
pref_hidden_negatives = get_negatives_until_explosion(pref_hidden_data)
pref_description_negatives = get_negatives_until_explosion(pref_description_data)
alt_hidden_negatives = get_negatives_until_explosion(alt_hidden_data)
alt_description_negatives = get_negatives_until_explosion(alt_description_data)
hidden_description_negatives = get_negatives_until_explosion(hidden_description_data)

In [ ]:
pref_alt_negatives

In [ ]:
subset_df = pref_alt_negatives[['?a', 'sampled_negatives']]

In [ ]:
subset_df

In [ ]:
subset_df_exploded = subset_df.explode('sampled_negatives')

In [ ]:
len(subset_df['sampled_negatives'].iloc[0]), subset_df['?a'].iloc[0]

In [ ]:
subset_df_exploded[subset_df_exploded['?a'] == 'http://data.europa.eu/esco/skill/0005c151-5b5a-4a66-8aac-60e734beb1ab']

In [ ]:
subset_df_exploded.drop_duplicates(inplace=True)

In [ ]:
subset_df_exploded

In [ ]:
def explode_and_append(df_with_negative, df_to_append):

    df_negative = df_with_negative.copy(deep=True)
    df_append = df_to_append.copy(deep=True)
    df_append.rename(columns={df_append.columns[0]:'sampled_negatives'}, inplace=True)
    df_append.rename(columns={df_append.columns[1]:df_append.columns[1]+'_negative'}, inplace=True)
    df_negative.drop(columns = ['negative_uris', 'counts'], inplace=True)
    df_negatives_exploded = df_negative.explode('sampled_negatives')
    # combined_df = skills_copy.merge(df1, on=['?a'], how='left').merge(df2, on=['?a'], how='left')
    df_appended = df_negatives_exploded.merge(df_append, on=['sampled_negatives'], how='left')
    df_appended.drop(columns=['sampled_negatives', '?a'], inplace=True)
    return df_appended

In [ ]:
pref_alt_exploded = explode_and_append(pref_alt_negatives, MemConcept_SkillsHier_altLabel)
pref_hidden_exploded = explode_and_append(pref_hidden_negatives, MemConcept_SkillsHier_hiddenLabel)
pref_description_exploded = explode_and_append(pref_description_negatives, MemConcept_SkillsHier_description)
alt_hidden_exploded = explode_and_append(alt_hidden_negatives, MemConcept_SkillsHier_hiddenLabel)
alt_description_exploded = explode_and_append(alt_description_negatives, MemConcept_SkillsHier_description)
hidden_description_exploded = explode_and_append(hidden_description_negatives, MemConcept_SkillsHier_description)

In [ ]:
full_data = [pref_alt_exploded,pref_hidden_exploded,pref_description_exploded,alt_hidden_exploded,alt_description_exploded,hidden_description_exploded]

In [ ]:
for df in full_data:
    df.columns = ['anchor', 'positive_data', 'negative_data']

In [ ]:
all_negative_data_merged = pd.concat(full_data)

In [ ]:
all_negative_data_merged.shape

In [ ]:
all_negative_data_merged.dropna(inplace=True)

In [ ]:
all_negative_data_merged.shape

In [ ]:
all_negative_data_merged.drop_duplicates(inplace=True)

In [ ]:
all_negative_data_merged.memory_usage(index=True).sum()

In [ ]:
file_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/{LANGUAGE}/skills_labels_triplet_pairs.csv'

In [ ]:
all_negative_data_merged.to_csv(file_dir, sep='\t')

In [ ]:
raise